In [7]:
!pip install category_encoders


In [8]:
!pip install optuna

In [9]:
!pip install lime


In [10]:
from google.colab import files
uploaded = files.upload()


Saving data.txt to data (1).txt


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
import optuna
import math
from tqdm import tqdm
from scipy.stats import norm
import cv2
import lime
import lime.lime_tabular

# Suppress optuna logging
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Default data path
DEFAULT_DATA_PATH = 'data.txt'

# Constants
FAMOUS_ARTISTS = [
    'Pablo Picasso', 'Andy Warhol', 'Gustav Klimt', 'Paul Cezanne',
    'Edvard Munch', 'Vincent van Gogh', 'Mark Rothko', 'Leonardo da Vinci'
]

MATERIALS = [
    'oil on canvas', 'acrylic on canvas', 'watercolor', 'charcoal', 'pastel',
    'ink on paper', 'mixed media on canvas', 'oil on panel', 'acrylic on wood',
    'oil on board', 'bronze', 'plaster', 'gouache', 'tempera on board', 'ceramic',
    'wood', 'metal', 'stone', 'fabric', 'mixed media', 'oil and acrylic',
    'acrylic and collage on canvas', 'oil on paper', 'oil on wood', 'oil on cardboard',
    'acrylic on paper', 'gouache and watercolor on paper', 'ink and watercolor on paper',
    'oil and mixed media on canvas', 'acrylic and spray paint on canvas', 'oil on copper',
    'oil on plywood'
]


REGIONS = {
    # Define your region mapping here
    'Swiss': 'Europe', 'French': 'Europe', 'American': 'North America', 'European': 'Europe',
    'Danish': 'Europe', 'Chinese': 'Asia', 'Taiwanese': 'Asia', 'British': 'Europe',
    'Iranian': 'Asia', 'Dutch': 'Europe', 'German': 'Europe', 'Norwegian': 'Europe',
    'Swedish': 'Europe', 'Finnish': 'Europe', 'Hungarian': 'Europe', 'Italian': 'Europe',
    'Filipino': 'Asia', 'Spanish': 'Europe', 'Croatian': 'Europe', 'Polish': 'Europe',
    'Argentine': 'South America', 'Chilean': 'South America', 'Mexican': 'North America',
    'Azerbaijani': 'Asia', 'Russian': 'Europe', 'Belgian': 'Europe', 'Malaysian': 'Asia',
    'Iraqi': 'Asia', 'Persian': 'Asia', 'Austrian': 'Europe', 'Australian': 'Oceania',
    'Lebanese': 'Asia', 'Palestinian': 'Asia', 'Egyptian': 'Africa', 'Syrian': 'Asia',
    'Algerian': 'Africa', 'Turkish': 'Asia', 'Mozambican': 'Africa', 'Javanese': 'Asia',
    'Armenian': 'Asia', 'Japanese': 'Asia', 'Israeli': 'Asia', 'Bangladeshi': 'Asia',
    'Indonesian': 'Asia', 'Cuban': 'North America', 'Haitian': 'North America',
    'NewZealand': 'Oceania', 'Indian': 'Asia', 'Moroccan': 'Africa', 'Libyan': 'Africa',
    'Georgian': 'Asia', 'Yugoslav': 'Europe', 'Lithuanian': 'Europe', 'SouthAfrican': 'Africa',
    'Guatemalan': 'North America', 'Flemish': 'Europe', 'Czech': 'Europe', 'Greek': 'Europe',
    'Canadian': 'North America', 'Uruguayan': 'South America', 'Colombian': 'South America',
    'Venezuelan': 'South America', 'Scottish': 'Europe', 'NorthAmerican': 'North America',
    'Estonian': 'Europe', 'Irish': 'Europe', 'Nigerian': 'Africa', 'Balinese': 'Asia',
    'Romanian': 'Europe', 'Portuguese': 'Europe', 'Pakistani': 'Asia', 'Korean': 'Asia',
    'PuertoRican': 'North America', 'Scandinavian': 'Europe', 'Emirati': 'Asia',
    'Thai': 'Asia', 'Serbian': 'Europe', 'Sudanese': 'Africa', 'Dominican': 'North America',
    'LatinAmerican': 'South America', 'Peruvian': 'South America', 'SouthAmerican': 'South America',
    'Welsh': 'Europe', 'Singaporean': 'Asia', 'Brazilian': 'South America', 'Cambodian': 'Asia',
    'NativeAmerican': 'North America', 'Continental': 'Europe', 'Senegalese': 'Africa',
    'Qatari': 'Asia', 'Jordanian': 'Asia', 'Beninese': 'Africa', 'Asian': 'Asia',
    'Mongolian': 'Asia', 'Tanzanian': 'Africa', 'Vietnamese': 'Asia', 'Latvian': 'Europe',
    'Zimbabwean': 'Africa', 'Ecuadorian': 'South America', 'Ghanaian': 'Africa',
    'Tunisian': 'Africa', 'Luxembourger': 'Europe', 'Slovak': 'Europe', 'Uzbekistani': 'Asia',
    'Congolese': 'Africa', 'Bahamian': 'North America', 'Bulgarian': 'Europe',
    'CostaRican': 'North America', 'Bolivian': 'South America', 'Albanian': 'Europe',
    'Nicaraguan': 'North America', 'Ukrainian': 'Europe', 'African': 'Africa',
    'Bahraini': 'Asia'
}


In [12]:
def extract_image_features(image_path):
    """Extract comprehensive image features including the dominant color and face count"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("Could not read image")

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        brightness = np.mean(img_rgb)
        unique_colors = len(np.unique(img_rgb.reshape(-1, 3), axis=0))
        ratio_unique_colors = unique_colors / (img_rgb.shape[0] * img_rgb.shape[1])
        threshold_black = np.sum(img_gray < 30) / (img_gray.shape[0] * img_gray.shape[1])
        high_brightness = np.sum(img_gray > 200) / (img_gray.shape[0] * img_gray.shape[1])
        low_brightness = np.sum(img_gray < 50) / (img_gray.shape[0] * img_gray.shape[1])
        edges = cv2.Canny(img_gray, 100, 200)
        edge_percentage = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
        corners = cv2.goodFeaturesToTrack(img_gray, maxCorners=100, qualityLevel=0.01, minDistance=10)
        corner_percentage = len(corners) / (img_gray.shape[0] * img_gray.shape[1]) if corners is not None else 0

        cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        face_cascade = cv2.CascadeClassifier(cascade_path)

        faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))
        face_count = len(faces) if len(faces) > 0 else 0

        pixels = np.float32(img_rgb).reshape(-1, 3)
        n_colors = 5
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
        _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        _, counts = np.unique(labels, return_counts=True)

        dominant_color = palette[np.argmax(counts)].astype(int)

        colors = {
            'red': [255, 0, 0], 'green': [0, 255, 0], 'blue': [0, 0, 255],
            'yellow': [255, 255, 0], 'white': [255, 255, 255], 'black': [0, 0, 0],
            'brown': [165, 42, 42], 'maroon': [128, 0, 0], 'orange': [255, 165, 0],
            'purple': [128, 0, 128], 'pink': [255, 192, 203], 'gray': [128, 128, 128]
        }

        dominant_color_name = min(colors.keys(), key=lambda x: np.linalg.norm(np.array(colors[x]) - dominant_color))

        return {
            'brightness': float(brightness),
            'ratioUniqueColors': float(ratio_unique_colors),
            'thresholdBlackPerc': float(threshold_black),
            'highbrightnessPerc': float(high_brightness),
            'lowbrightnessPerc': float(low_brightness),
            'CornerPer': float(corner_percentage),
            'EdgePer': float(edge_percentage),
            'FaceCount': int(face_count),
            'dominantColor': str(dominant_color_name)
        }

    except Exception as e:
        print(f"Error processing image: {e}")
        return {
            'brightness': 0.0, 'ratioUniqueColors': 0.0, 'thresholdBlackPerc': 0.0,
            'highbrightnessPerc': 0.0, 'lowbrightnessPerc': 0.0, 'CornerPer': 0.0,
            'EdgePer': 0.0, 'FaceCount': 0, 'dominantColor': 'unknown'
        }


In [13]:
def clean_and_prepare_data(df):
    """Clean and prepare the dataset for training"""
    df_clean = df.copy()

    if 'soldtime' in df_clean.columns:
        df_clean.drop(columns=['soldtime'], inplace=True)

    required_columns = ['year', 'height', 'width', 'artist', 'material', 'price']
    missing_cols = [col for col in required_columns if col not in df_clean.columns]
    if missing_cols:
        print(f"Missing required columns: {missing_cols}")
        return None, None, None, None, None

    numeric_cols = ['year', 'height', 'width', 'price']
    for col in numeric_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        col_median = df_clean[col].median(skipna=True)
        df_clean[col] = df_clean[col].fillna(col_median).astype('float32' if col in ['height', 'width', 'price'] else 'int16')

    df_clean['log_price'] = np.log1p(df_clean['price'])
    df_clean['area'] = df_clean['height'] * df_clean['width']
    df_clean['log_area'] = np.log1p(df_clean['area'])
    df_clean['aspect_ratio'] = df_clean['height'] / df_clean['width']
    df_clean['is_famous_artist'] = df_clean['artist'].isin(FAMOUS_ARTISTS).astype('int8')
    df_clean['artwork_age'] = 2025 - df_clean['year']

    encoder_artist = TargetEncoder(cols=['artist'])
    encoder_material = TargetEncoder(cols=['material'])
    df_clean['artist_encoded'] = encoder_artist.fit_transform(df_clean[['artist']], df_clean['log_price'])['artist']
    df_clean['material_encoded'] = encoder_material.fit_transform(df_clean[['material']], df_clean['log_price'])['material']

    if 'country' in df_clean.columns:
        df_clean['region'] = df_clean['country'].map(REGIONS).fillna('Other')
        encoder_region = TargetEncoder(cols=['region'])
        df_clean['region_encoded'] = encoder_region.fit_transform(df_clean[['region']], df_clean['log_price'])['region']
    else:
        df_clean['region_encoded'] = 0
        encoder_region = TargetEncoder(cols=['region'])  # Dummy encoder for consistency

    feature_columns = [
        'material_encoded', 'artist_encoded', 'region_encoded', 'year', 'log_area',
        'aspect_ratio', 'is_famous_artist', 'artwork_age'
    ]

    image_features = ['brightness', 'ratioUniqueColors', 'thresholdBlackPerc',
                      'highbrightnessPerc', 'lowbrightnessPerc', 'CornerPer',
                      'EdgePer', 'FaceCount']
    for feat in image_features:
        if feat in df_clean.columns:
            df_clean[feat] = pd.to_numeric(df_clean[feat], errors='coerce').fillna(0)
            feature_columns.append(feat)

    y = df_clean['log_price'].values
    X = df_clean[feature_columns].values

    return X, y, (encoder_artist, encoder_material, encoder_region), feature_columns, df_clean

def introduce_noise(X, noise_level=0.1):
    """Introduce Gaussian noise to numeric features"""
    X_noisy = X.copy()
    feature_indices = {name: i for i, name in enumerate([
        'material_encoded', 'artist_encoded', 'region_encoded', 'brightness', 'ratioUniqueColors',
        'thresholdBlackPerc', 'highbrightnessPerc', 'lowbrightnessPerc', 'CornerPer', 'EdgePer',
        'FaceCount', 'is_famous_artist', 'year', 'log_area', 'aspect_ratio', 'artwork_age'
    ])}

    continuous_features = ['brightness', 'ratioUniqueColors', 'thresholdBlackPerc', 'highbrightnessPerc',
                           'lowbrightnessPerc', 'CornerPer', 'EdgePer', 'FaceCount', 'log_area', 'aspect_ratio',
                           'artwork_age']  # Ensure no leading spaces
    for feature in continuous_features:
        index = feature_indices[feature]
        noise = np.random.normal(0, noise_level, X_noisy[:, index].shape)
        X_noisy[:, index] += noise
    return X_noisy

In [28]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    """Objective function for hyperparameter optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1, 10),
        'random_state': 42
    }
    model = XGBRegressor(**params, eval_metric="mae", early_stopping_rounds=10)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    y_pred = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, y_pred)

def train_model(data_path=DEFAULT_DATA_PATH):
    """Train the price prediction model."""
    print("Loading and preparing data...")
    df = pd.read_table(data_path, low_memory=False)

    # Unpack the values returned by clean_and_prepare_data
    X, y, encoders, feature_columns, df_clean = clean_and_prepare_data(df)

    # Check if the data preparation was successful
    if X is None or y is None:
        print("Data preparation failed. Exiting.")
        return None, None, None, None

    # Introduce noise to the features
    X_noisy = introduce_noise(X, noise_level=0.1)

    # Split the data into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X_noisy, y, test_size=0.2, random_state=42)

    print("Optimizing hyperparameters...")
    study = optuna.create_study(direction='minimize')

    # K-Fold Cross Validation for Optuna
    kf = KFold(n_splits=3, shuffle=True, random_state=42)  # Reduced splits to speed up training

    for train_index, val_index in tqdm(kf.split(X_train), total=3, desc="Cross-Validation"):
        X_kf_train, X_kf_val = X_train[train_index], X_train[val_index]
        y_kf_train, y_kf_val = y_train[train_index], y_train[val_index]

        # Optimize hyperparameters using the objective function
        study.optimize(lambda trial: objective(trial, X_kf_train, y_kf_train, X_kf_val, y_kf_val), n_trials=30)  # Reduced trials

    best_params = study.best_params
    model = XGBRegressor(**best_params, eval_metric="mae", early_stopping_rounds=10)  # Move early_stopping_rounds here

    # Final model fit on the entire training set
    model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              verbose=False)  # Silent training

    # Calculate model performance
    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    mae = mean_absolute_error(y_valid, y_pred)

    print(f"\nModel Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")

    return model, encoders, np.std(y_valid - y_pred), feature_columns, df_clean


In [29]:
def print_feature_importances(model, feature_columns):
    """Print feature importances from the model"""
    importances = model.feature_importances_
    feature_importance_df = pd.DataFrame({'Feature': feature_columns, 'Importance': importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    print("\nFeature Importances:")
    print(feature_importance_df)

In [30]:
def explain_with_lime(model, X_train, feature_columns, inputs):
    """Use LIME to explain the model's predictions"""
    explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_columns, mode='regression')

    # Convert inputs dictionary to DataFrame
    input_df = pd.DataFrame([inputs], columns=feature_columns)

    # Explain the prediction
    exp = explainer.explain_instance(input_df.values[0], model.predict, num_features=10)
    exp.show_in_notebook(show_table=True)

In [32]:
xgb_model, encoders, std_dev, feature_columns, df = train_model(DEFAULT_DATA_PATH)


Loading and preparing data...
Optimizing hyperparameters...


Cross-Validation: 100%|██████████| 3/3 [05:21<00:00, 107.15s/it]



Model Performance:
R² Score: 0.8330
Mean Absolute Error: 0.6074


In [34]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [36]:
!pip uninstall -y numpy catboost
!pip install --no-cache-dir numpy catboost


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: catboost 1.2.7
Uninstalling catboost-1.2.7:
  Successfully uninstalled catboost-1.2.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 196.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 150.1 MB/s eta 0:00:00


In [1]:
from catboost import CatBoostRegressor


In [3]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    """Objective function for hyperparameter optimization"""
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'depth': trial.suggest_int('depth', 3, 10),  # Changed parameter name
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),  # Different parameter
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),  # Different parameter name
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),  # Different parameter name
        'random_seed': 42,  # Changed parameter name
        'verbose': False
    }

    # Changed model class and parameters
    model = CatBoostRegressor(**params, eval_metric='MAE', early_stopping_rounds=10)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    y_pred = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, y_pred)

def train_model(data_path='data.txt'):
    """Train the price prediction model."""
    print("Loading and preparing data...")
    df = pd.read_table(data_path, low_memory=False)

    # Unpack the values returned by clean_and_prepare_data
    X, y, encoders, feature_columns, df_clean = clean_and_prepare_data(df)

    if X is None or y is None:
        print("Data preparation failed. Exiting.")
        return None, None, None, None

    X_noisy = introduce_noise(X, noise_level=0.1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_noisy, y, test_size=0.2, random_state=42)

    print("Optimizing hyperparameters...")
    study = optuna.create_study(direction='minimize')

    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    for train_index, val_index in tqdm(kf.split(X_train), total=3, desc="Cross-Validation"):
        X_kf_train, X_kf_val = X_train[train_index], X_train[val_index]
        y_kf_train, y_kf_val = y_train[train_index], y_train[val_index]

        study.optimize(lambda trial: objective(trial, X_kf_train, y_kf_train, X_kf_val, y_kf_val), n_trials=30)

    best_params = study.best_params
    # Changed model class and parameters
    model = CatBoostRegressor(**best_params, eval_metric='MAE', early_stopping_rounds=10)

    model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              verbose=False)

    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    mae = mean_absolute_error(y_valid, y_pred)

    print(f"\nModel Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")

    return model, encoders, np.std(y_valid - y_pred), feature_columns, df_clean


In [14]:
model, encoders, std_dev, feature_columns, df_clean = train_model('data.txt')


Loading and preparing data...
Optimizing hyperparameters...


Cross-Validation: 100%|██████████| 3/3 [12:51<00:00, 257.02s/it]



Model Performance:
R² Score: 0.8327
Mean Absolute Error: 0.6136


In [27]:
import numpy as np
import pandas as pd
import optuna
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from lightgbm import LGBMRegressor, early_stopping

def objective(trial, X_train, y_train, X_valid, y_valid):
    """Objective function for hyperparameter optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'random_state': 42,
        'verbosity': -1

    }

    model = LGBMRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[early_stopping(10)])
    y_pred = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, y_pred)

def train_model(data_path='data.txt'):
    """Train the price prediction model."""
    print("Loading and preparing data...")
    df = pd.read_table(data_path, low_memory=False)

    X, y, encoders, feature_columns, df_clean = clean_and_prepare_data(df)

    if X is None or y is None:
        print("Data preparation failed. Exiting.")
        return None, None, None, None

    X_noisy = introduce_noise(X, noise_level=0.1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_noisy, y, test_size=0.2, random_state=42)

    print("Optimizing hyperparameters...")
    study = optuna.create_study(direction='minimize')

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    for train_index, val_index in tqdm(kf.split(X_train), total=3, desc="Cross-Validation"):
        X_kf_train, X_kf_val = X_train[train_index], X_train[val_index]
        y_kf_train, y_kf_val = y_train[train_index], y_train[val_index]

        study.optimize(lambda trial: objective(trial, X_kf_train, y_kf_train, X_kf_val, y_kf_val), n_trials=30)

    best_params = study.best_params
    model = LGBMRegressor(**best_params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[early_stopping(10)])

    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    mae = mean_absolute_error(y_valid, y_pred)

    print(f"\nModel Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")

    return model, encoders, np.std(y_valid - y_pred), feature_columns, df_clean


In [28]:
model, encoders, std_dev, feature_columns, df_clean = train_model('data.txt')


Loading and preparing data...
Optimizing hyperparameters...


Cross-Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Training until validation scores don't improve for 10 rounds


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Did not meet early stopping. Best iteration is:
[150]	valid_0's l2: 1.58979


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[281]	valid_0's l2: 0.725494


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Cross-Validation: 100%|██████████| 3/3 [04:09<00:00, 83.18s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[349]	valid_0's l2: 0.714524


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Model Performance:
R² Score: 0.8356
Mean Absolute Error: 0.6000


In [29]:
import numpy as np
import pandas as pd
import optuna
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

def objective(trial, X_train, y_train, X_valid, y_valid):
    """Objective function for hyperparameter optimization"""
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'random_state': 42
    }

    model = DecisionTreeRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, y_pred)

def train_model(data_path='data.txt'):
    """Train the price prediction model."""
    print("Loading and preparing data...")
    df = pd.read_table(data_path, low_memory=False)

    X, y, encoders, feature_columns, df_clean = clean_and_prepare_data(df)

    if X is None or y is None:
        print("Data preparation failed. Exiting.")
        return None, None, None, None

    X_noisy = introduce_noise(X, noise_level=0.1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_noisy, y, test_size=0.2, random_state=42)

    print("Optimizing hyperparameters...")
    study = optuna.create_study(direction='minimize')

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    for train_index, val_index in tqdm(kf.split(X_train), total=3, desc="Cross-Validation"):
        X_kf_train, X_kf_val = X_train[train_index], X_train[val_index]
        y_kf_train, y_kf_val = y_train[train_index], y_train[val_index]

        study.optimize(lambda trial: objective(trial, X_kf_train, y_kf_train, X_kf_val, y_kf_val), n_trials=30)

    best_params = study.best_params
    model = DecisionTreeRegressor(**best_params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    mae = mean_absolute_error(y_valid, y_pred)

    print(f"\nModel Performance:")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")

    return model, encoders, np.std(y_valid - y_pred), feature_columns, df_clean


In [30]:
model, encoders, std_dev, feature_columns, df_clean = train_model('data.txt')


Loading and preparing data...
Optimizing hyperparameters...


Cross-Validation: 100%|██████████| 3/3 [00:41<00:00, 13.98s/it]



Model Performance:
R² Score: 0.7855
Mean Absolute Error: 0.6709
